# Agentic retrieval using SharePoint Remote Knowledge Source

Use this notebook to create an agentic retrieval pipeline that queries SharePoint Online at runtime without ingestion.

In this walkthrough, you will:

+ Configure SharePoint authentication
+ Create a knowledge base for runtime queries
+ Create a Foundry agent to determine when queries are needed
+ Query SharePoint documents in real-time
+ Apply filters and search scopes

**Note**: Remote SharePoint sources query documents at runtime without pre-indexing.

**Important**: Remote SharePoint knowledge sources currently require REST API calls as the Python SDK support is limited. This notebook demonstrates the pattern using REST API with Python requests library.

## Prerequisites

+ Azure AI Search, basic tier or higher.
+ SharePoint Online site with documents.
+ Azure OpenAI and Azure AI Foundry project.
+ A deployment of gpt-4o-mini in your Foundry project.

We recommend creating a virtual environment to run this sample code.

## Install required packages

In [ ]:
%pip install azure-search-documents==11.7.0b2
%pip install azure-identity
%pip install azure-ai-projects
%pip install azure-mgmt-cognitiveservices
%pip install python-dotenv
%pip install requests

## Load connections

In [ ]:
from dotenv import load_dotenv
from azure.identity import DefaultAzureCredential
from azure.mgmt.core.tools import parse_resource_id
import os
import requests
import json

load_dotenv(override=True)

project_endpoint = os.environ["PROJECT_ENDPOINT"]
project_resource_id = os.environ["PROJECT_RESOURCE_ID"]
project_connection_name = os.getenv("PROJECT_CONNECTION_NAME", "sharepointremoteconnection")
agent_model = os.getenv("AGENT_MODEL", "gpt-4o-mini")
agent_name = os.getenv("AGENT_NAME", "sharepoint-remote-agent")
endpoint = os.environ["AZURE_SEARCH_ENDPOINT"]
search_api_key = os.environ["AZURE_SEARCH_API_KEY"]
credential = DefaultAzureCredential()
base_name = os.getenv("AZURE_SEARCH_AGENT_NAME", "sharepoint-remote-base")
sharepoint_site_url = os.getenv("SHAREPOINT_SITE_URL", "https://yourtenant.sharepoint.com/sites/yoursite")
api_version = "2025-11-01-preview"

parsed_resource_id = parse_resource_id(project_resource_id)
subscription_id = parsed_resource_id['subscription']
resource_group = parsed_resource_id['resource_group']
account_name = parsed_resource_id['name']
project_name = parsed_resource_id['child_name_1']

## Create a knowledge base

For remote SharePoint, we create a knowledge base without pre-defined knowledge sources. The SharePoint source is specified at query time.

**Note**: Currently using REST API as SDK support for remote SharePoint is limited.

In [ ]:
url = f"{endpoint}/knowledgeBases/{base_name}?api-version={api_version}"

headers = {
    "api-key": search_api_key,
    "Content-Type": "application/json"
}

body = {
    "name": base_name,
    "description": "Knowledge base with SharePoint remote access",
    "knowledgeSources": [],
    "outputMode": "extractiveData",
    "retrievalReasoningEffort": {"kind": "minimal"}
}

response = requests.put(url, headers=headers, json=body)
print(f"Status: {response.status_code}")
if response.status_code in [200, 201]:
    print(f"Knowledge base '{base_name}' created or updated successfully")
    mcp_endpoint = f"{endpoint}/knowledgebases/{base_name}/mcp?api-version={api_version}"
else:
    print(f"Error: {response.text}")

## Create an MCP Tool Connection

In [ ]:
from azure.mgmt.cognitiveservices import CognitiveServicesManagementClient
from azure.mgmt.cognitiveservices.models import (
    ConnectionPropertiesV2BasicResource,
    CustomKeysConnectionProperties,
    CustomKeys
)

mgmt_client = CognitiveServicesManagementClient(credential, subscription_id)
resource = mgmt_client.project_connections.create(
    resource_group_name=resource_group,
    account_name=account_name,
    project_name=project_name,
    connection_name=project_connection_name,
    connection=ConnectionPropertiesV2BasicResource(
        properties=CustomKeysConnectionProperties(
            category="RemoteTool",
            target=mcp_endpoint,
            is_shared_to_all=True,
            metadata={"ApiType": "Azure"},
            credentials=CustomKeys(
                keys={"api-key": search_api_key}
            )
        )
    )
)

print(f"Connection '{resource.name}' created or updated successfully.")

## Create an Azure AI Agent

In [ ]:
from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import PromptAgentDefinition, MCPTool

project_client = AIProjectClient(endpoint=project_endpoint, credential=credential)

instructions = """
A Q&A agent that can answer questions from SharePoint documents.
Always provide references to the documents used to answer questions.
If you do not have the answer, respond with "I don't know".
"""

mcp_kb_tool = MCPTool(
    server_label="knowledge-base",
    server_url=mcp_endpoint,
    require_approval="never",
    allowed_tools=["knowledge_base_retrieve"],
    project_connection_id=project_connection_name
)

agent = project_client.agents.create_version(
    agent_name=agent_name,
    definition=PromptAgentDefinition(
        model=agent_model,
        instructions=instructions,
        tools=[mcp_kb_tool]
    )
)

print(f"AI agent '{agent_name}' created or updated successfully")

## Query SharePoint documents

**Note**: Remote SharePoint querying requires runtime authorization. Ensure you have appropriate SharePoint permissions.

In [ ]:
openai_client = project_client.get_openai_client()

conversation = openai_client.conversations.create()

response = openai_client.responses.create(
    conversation=conversation.id,
    input="What are the latest project updates in SharePoint?",
    extra_body={"agent": {"name": agent.name, "type": "agent_reference"}},
)

print(f"Response: {response.output_text}")

## Clean up resources

### Delete the agent

In [ ]:
project_client.agents.delete_version(agent.name, agent.version)
print(f"AI agent '{agent.name}' version '{agent.version}' deleted successfully")

### Delete the knowledge base

In [ ]:
url = f"{endpoint}/knowledgeBases/{base_name}?api-version={api_version}"
response = requests.delete(url, headers=headers)
print(f"Knowledge base delete status: {response.status_code}")